In [2]:
from funcy import *
from probdist import *

In [3]:
import numpy as np

# Motivation

The goal of the notebook is to prototype code for constructing a (locally) optimal channel code given several parameters:
 - some specification of a channel function $p(Y^* | X^*)$ with attendant descriptions of the set of source symbols $X$ and the set of channel symbols $Y$
 - some specification of a distribution over messages $p(W)$
 - some information about the nature of the channel code to construct (e.g. a fixed-length block code with block length $n$).

# Toy model

## BSC with erasure

In [15]:
Xs = {'0','1'}
Ys = {'0','1', 'ε'}

alpha = 0.1
beta = 1 - alpha

pY_X = {'0':{'0':beta,
             '1':alpha/2,
             'ε':alpha/2},
        '1':{'0':alpha/2,
             '1':beta,
             'ε':alpha/2}}

In [16]:
Xs_OH = seqsToOneHotMap(Xs)
Xs_OH

Ys_OH = seqsToOneHotMap(Ys)
Ys_OH

{'0': array([1., 0.]), '1': array([0., 1.])}

{'0': array([1., 0., 0.]), '1': array([0., 1., 0.]), 'ε': array([0., 0., 1.])}

In [22]:
pY_X_np = condDistFamilyToNP(pY_X)
pY_X_np
H_np(pY_X_np) #conditional entropies
H_np(pY_X_np, distToNP(Uniform(Xs))) #expected conditional entropy

array([[0.9 , 0.05],
       [0.05, 0.9 ],
       [0.05, 0.05]])

array([0.56899559, 0.56899559])

0.5689955935892812

In [20]:
pY_X_np @ Xs_OH['0'] #p(Y | X = 0)
H_np(pY_X_np @ Xs_OH['0']) #entropy of that distribution

array([0.9 , 0.05, 0.05])

0.5689955935892812